In [ ]:
# ==========================================
# 1. TÉLÉCHARGEMENT DU DATASET
# ==========================================
!pip install opendatasets
import opendatasets as od
import os

# Téléchargement (Il faudra saisir tes identifiants Kaggle)
dataset_url = "https://www.kaggle.com/datasets/iarunava/cell-images-for-detecting-malaria/data"
od.download(dataset_url)

# Le chemin vers les images téléchargées
data_dir = './cell-images-for-detecting-malaria/cell_images/cell_images'

# ==========================================
# 2. PRÉPARATION DES DONNÉES
# ==========================================
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import classification_report

# Transformation des images
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# Chargement depuis le dossier téléchargé
full_dataset = datasets.ImageFolder(root=data_dir, transform=transform)

# Division 80% Entraînement / 20% Test
train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_data, test_data = random_split(full_dataset, [train_size, test_size])

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# ==========================================
# 3. LE MODÈLE CNN (VOTRE CERVEAU ARTIFICIEL)
# ==========================================
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2), # 64 -> 32

            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)  # 32 -> 16
        )
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32 * 16 * 16, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.conv_layers(x)
        return self.fc_layers(x)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleNet().to(device)

# ==========================================
# 4. ENTRAÎNEMENT
# ==========================================
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCELoss()

print(f"Entraînement lancé sur {device}...")
for epoch in range(5):
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device).float().unsqueeze(1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f"Époque {epoch+1}/5 terminée.")

# ==========================================
# 5. ÉVALUATION FINALE:
# ==========================================
model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        outputs = model(images)
        preds = (outputs > 0.5).float()
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.numpy())

print("\n--- RAPPORT MÉDICAL ---")
print(classification_report(all_labels, all_preds, target_names=['Infecté', 'Sain']))